In [ ]:
import os
import sys
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import h2o
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
#============================================================
# 분석 환경 셋팅
#============================================================
sys.stdout.flush() #Python 메모리에 생성된 모든 객체 삭제(초기화)

#============================================================
# 작업 디렉토리 경로 확인
#============================================================
currentPath=os.getcwd()
print('Current working dir : %s' % currentPath)

#============================================================
# 기상 데이터 읽어오기 
#============================================================
ASOS_DATA = pd.read_csv(currentPath + '/ASOS_DATA.csv', encoding='euc-kr') #loading  weather data
GEO_DATA = pd.read_csv(currentPath + '/GEO_DATA.csv', encoding='euc-kr') #loading geographical data

#============================================================
# 불러온 데이터 구조 확인하기 
#============================================================
ASOS_DATA.info()
GEO_DATA.info()

In [ ]:
#=============================================================
# 테이블 결합 및 확인 
#=============================================================
DATA = pd.merge(ASOS_DATA, GEO_DATA, how='left', on='STNID')
print(ASOS_DATA.columns)
print(GEO_DATA.columns)
print(DATA.columns)
len(DATA)

In [ ]:
#=============================================================
# 데이터 타입 확인
#=============================================================
print('** (변경전) MOUNTAIN, Y 컬럼 데이터 타입 확인')
print('MOUNTAIN : ',DATA['MOUNTAIN'].dtype)
print('Y : ', DATA['Y'].dtype)

#=============================================================
# 데이터 타입 변환
#=============================================================  
DATA['MOUNTAIN']=DATA['MOUNTAIN'].astype('category')
DATA['Y']=DATA['Y'].astype('category')

#=============================================================
# 변환된 타입 확인
#=============================================================
print('** (변경후) MOUNTAIN, Y 컬럼 데이터 타입 확인')
print('MOUNTAIN : ', DATA['MOUNTAIN'].dtype)
print('Y : ', DATA['Y'].dtype)

In [ ]:
#=============================================================
# 요약 통계 한번에 보기
#=============================================================
DATA.describe(include='all')

In [ ]:
#============================================================
# 지점별 요약통계 보기 
#============================================================
grouped = DATA.groupby('STNID')
grouped.describe(include='all')

In [ ]:
#============================================================
# STN별 박스플롯 그리기
#============================================================
sns.set()
sns.set_style("ticks")
%matplotlib inline

col_bisque = ["#ffebcd"]
col_blue = ["#0000ff"]
col_gray = ["#a9a9a9"]
col_red = ["#ff0000"]

plt.figure(figsize=(12,6))
plt.title("Boxplot of STD_TS by STN_ID")
sns.boxplot(x="STNID", y="STD_TS", data=DATA, palette = sns.color_palette(col_bisque))
plt.show()

plt.figure(figsize=(12,6))
plt.title("Boxplot of AVG_TA by STN_ID")
sns.boxplot(x="STNID", y="AVG_TA", data=DATA, palette = sns.color_palette(col_blue))
plt.show()

plt.figure(figsize=(12,6))
plt.title("Boxplot of MAX_WS by STN_ID")
sns.boxplot(x="STNID", y="MAX_WS", data=DATA, palette = sns.color_palette(col_gray))
plt.show()

plt.figure(figsize=(12,6))
plt.title("Boxplot of MIN_TCA by STN_ID")
sns.boxplot(x="STNID", y="MIN_TCA", data=DATA, palette = sns.color_palette(col_red))
plt.show()

In [ ]:
#============================================================
# Histogram 그리기 
#============================================================
plt.figure(figsize=(12,6))
plt.title("Histogram of STD_TS")
plt.xlabel('STD_TS')
plt.ylabel('Freqeuncy')
plt.hist(DATA['STD_TS'].dropna(), color = col_bisque)
plt.show()

plt.figure(figsize=(12,6))
plt.title("Histogram of AVG_TA")
plt.xlabel('AVG_TA')
plt.ylabel('Freqeuncy')
plt.hist(DATA['AVG_TA'].dropna(), color = col_blue)
plt.show()

plt.figure(figsize=(12,6))
plt.title("Histogram of MAX_WS")
plt.xlabel('MAX_WS')
plt.ylabel('Freqeuncy')
plt.hist(DATA['MAX_WS'].dropna(), color = col_gray)
plt.show()

plt.figure(figsize=(12,6))
plt.title("Histogram of MIN_TCA")
plt.xlabel('MIN_TCA')
plt.ylabel('Freqeuncy')
plt.hist(DATA['MIN_TCA'].dropna(), color = col_red)
plt.show()

In [ ]:
#=============================================================
# 이상치 확인
#=============================================================
DATA.describe(include='all')

#=============================================================
# 예시 : 기온 관련 변수 이상치 처리 방법 (기준값 : -80 ~ 60℃)
#=============================================================  
DATA.at[(DATA['MIN_TA'] < -80) | (DATA['MIN_TA'] > 60), 'MIN_TA'] = np.nan
DATA.at[(DATA['MAX_TA'] < -80) | (DATA['MAX_TA'] > 60), 'MAX_TA'] = np.nan
DATA.at[(DATA['AVG_TA'] < -80) | (DATA['AVG_TA'] > 60), 'AVG_TA'] = np.nan
DATA.at[(DATA['STD_TA'] < -80) | (DATA['STD_TA'] > 60), 'STD_TA'] = np.nan
DATA.describe(include = 'all')

In [ ]:
#=============================================================
# 결측치 확인
#=============================================================
DATA.isnull().sum()

#=============================================================
# 예시 : 강수량(RN), 일조(SS), 일사(SI) 관련 변수를 0으로 치환
#=============================================================  
DATA['AVG_RN'] = DATA['AVG_RN'].fillna(0)
DATA['SUM_RN'] = DATA['SUM_RN'].fillna(0)
DATA['MIN_SS'] = DATA['MIN_SS'].fillna(0)
DATA['MAX_SS'] = DATA['MAX_SS'].fillna(0)
DATA['AVG_SS'] = DATA['AVG_SS'].fillna(0)
DATA['STD_SS'] = DATA['STD_SS'].fillna(0)
DATA['MIN_SI'] = DATA['MIN_SI'].fillna(0)
DATA['MAX_SI'] = DATA['MAX_SI'].fillna(0)
DATA['AVG_SI'] = DATA['AVG_SI'].fillna(0)
DATA['STD_SI'] = DATA['STD_SI'].fillna(0)

# 데이터 결측치 확인
DATA.isnull().sum()
DATA.isnull().sum().sum()

In [ ]:
#=============================================================
# 기온(최고-최저) 파생변수 생성
#=============================================================  
DATA['MMT_TA']=DATA['MAX_TA']-DATA['MIN_TA']

#=============================================================
# 현상 발생 월, 시간 파생변수 생성 및 타입 변환
#=============================================================
DATA['MONTH'] = DATA['TM'].str.slice(5,7)
DATA['HOUR'] = DATA['TM'].str.slice(11,13)

DATA['MONTH'] = DATA['MONTH'].astype('category')
DATA['HOUR'] = DATA['HOUR'].astype('category')

DATA.info()

In [ ]:
#=============================================================
# 로컬에 H2O 가상서버 설정하기
#=============================================================
h2o.init(max_mem_size = "4G", nthreads = 4)
h2o.remove_all()

In [ ]:
#=============================================================
# 데이터 분할
#=============================================================
DATA = DATA.drop(['TM', 'MIN_SI', 'MIN_SS'], axis=1)
data_hex = h2o.H2OFrame(DATA)

data_hex['Y']=data_hex['Y'].asfactor()
data_hex['MOUNTAIN']=data_hex['MOUNTAIN'].asfactor()
data_hex['MONTH']=data_hex['MONTH'].asfactor()
data_hex['HOUR']=data_hex['HOUR'].asfactor()

train_data, test_data = data_hex.split_frame([0.8], seed=1234)

#=============================================================
# 분할된 데이터 셋 확인
#=============================================================
print(len(data_hex))
print(len(train_data))
print(len(test_data))

In [ ]:
#=============================================================
# 변수들간 다중공선성 제거
#=============================================================
# 상관계수를 구하기 전 범주형 변수 제외
X = DATA.drop(['STNID', 'Y', 'MOUNTAIN', 'MONTH', 'HOUR'], axis=1)
X.info()

# 변수들 간의 상관계수 산출
corr = X.corr()
Abs_corr = pd.DataFrame(abs(corr))
Abs_corr['variable']=Abs_corr.index
Abs_corr.reset_index(drop=True, inplace=True)
Abs_corr.info()

#=============================================================
# variable importance 산출하기 
#=============================================================
new_data_hex = train_data.drop(['STNID','Y'], axis=1)

xList = new_data_hex.columns
y = "Y"

gbm = H2OGradientBoostingEstimator(ntrees=50, max_depth=5, seed=1234)
gbm.train(x = xList, y = y, training_frame = data_hex)
gbm

varimp = gbm.varimp(True)
varimp = varimp[varimp.variable != 'MOUNTAIN']
varimp = varimp[varimp.variable != 'MONTH']
varimp = varimp[varimp.variable != 'HOUR']

varimp

In [ ]:
#=============================================================
# 다중공선성 제거
#=============================================================

finalvar = varimp 
j=0

while j < len(finalvar.variable):
    tmp = Abs_corr[(Abs_corr.variable != 'y') & (Abs_corr.variable != finalvar.variable[j])].loc[:,['variable',finalvar.variable[j]]]
    tmp.columns = ['variable', 'Pearson']
    tmp.sort_values(['Pearson'], axis=0, ascending=False, inplace=True)
    tmp = tmp[tmp.Pearson > 0.45]

    finalvar = pd.merge(finalvar, tmp, how='left', on='variable')
    finalvar = finalvar.loc[finalvar.isnull()['Pearson'], :]
    finalvar = finalvar.reset_index(drop=True)
    finalvar = finalvar.drop('Pearson',axis=1)
    finalvar.sort_values(['scaled_importance'], axis=0, ascending=False, inplace=True)
    j = j + 1

finalvar #최종선택변수

In [ ]:
#=============================================================
# 하이퍼파라미터 최적화 
#=============================================================
varList = list(finalvar['variable'])
extList = ['MOUNTAIN', 'MONTH', 'HOUR']
varList = varList + extList
varList
y = "Y"

# 하이퍼파라미터 조합만들기
hyper_params = {'sample_rate': [0.3, 0.4],
                'max_depth': [18, 20, 25],
                'ntrees': [25, 50]}

# 조합 모형 돌리기
m = H2OGridSearch(H2OGradientBoostingEstimator, grid_id = 'gbm_grid', hyper_params = hyper_params)
m.train(x = varList, y = y, training_frame = train_data)

# AUC가 높은 순으로 정렬하기
sorted_grid = m.get_grid(sort_by = 'auc', decreasing=True)
print('===== sorted_grid =====')
print(sorted_grid)

# 베스트 모형 선택
best_model = h2o.get_model(sorted_grid.model_ids[0])
print('===== best_model =====')
print(best_model)

In [ ]:
#=============================================================
# 모형 성능 및 예측력 검증
#=============================================================
# 최종 선택 모형 구축
gbm_fit = H2OGradientBoostingEstimator(ntrees=50, max_depth=25, sample_rate = 0.4, seed=1234)
gbm_fit.train(x = varList, y = y, training_frame = train_data)

# 모형 성능 검증
gbm_fit.auc()
gbm_fit.confusion_matrix()

per_gbm = gbm_fit.model_performance(test_data = test_data)
per_gbm.auc()
per_gbm.confusion_matrix()

In [ ]:
#=============================================================
# 지점별 예측력 검증
#=============================================================
# STN별 리스트 생성
STNList = DATA['STNID'].unique()

# 결과 리스트 생성
auc = list()
confusion = list()

# 지점별 데이터 셋 분할, 모형 구축, 예측력 검증
for i in range(len(STNList)):
    stn = STNList[i]
    print("STNID : ", stn, "...computing")
    train = data_hex[data_hex['STNID'] != int(stn), :]
    valid = data_hex[data_hex['STNID'] == int(stn), :]
    
    m = H2OGradientBoostingEstimator(ntrees=50, max_depth=25, sample_rate = 0.4, seed=1)
    m.train(x = varList, y = y, training_frame = train)
    
    per_m = m.model_performance(test_data = valid)

    auc_df = pd.DataFrame({'STNID':[stn], '':[per_m.auc()]})
    auc.append(auc_df)
    
    confusion_df = pd.DataFrame({'STNID':[stn], '':[per_m.confusion_matrix()]})
    confusion.append(confusion_df)

# STN별 교차검증 결과     
auc_stnid = pd.concat(auc)
confusion_stnid = pd.concat(confusion)

print(auc_stnid)
print(confusion_stnid)